In [102]:
import numpy as np
import pandas as pd
import requests
import json
import re

In [24]:
import dateutil
from dateutil.parser import parse
import datetime

getting weather *measurements* datapoints (for solar irradiance) from NREL SRRL BMS through the API:<br>
https://internal-apis.nrel.gov/intelligentcampus/hisRead?id=@p:nrel:r:20ed5df2-2c0e126b&range="2018-08-31%2c2018-09-02"

In the above request, note the manner in which the date range is specified.

to plug in the ID of the specific measurement (Dry Bulb Temperature in the following example) in the above link:<br>
NREL => Historian => Weather => NREL SRRL BMS => Dry Bulb Temperature => Aspects => Folio => ID

List of ids:
    - Energy Consumption => unit kwh => @p:nrel:r:225918db-bfbda16a
    - Relative Humidity (RH) => unit %RH  => @p:nrel:r:20ed5e0a-275dbdc2
    - Barometric Pressure (BP) => unit mbar  => @p:nrel:r:20ed5e0a-53e174aa
    - Dry Bulb Temperature (DBT) => unit degree Far => @p:nrel:r:20ed5e0a-fe755c80
    - Global Horizontal Irradiance (GHI) => unit watss/m2_irr => @p:nrel:r:20ed5df2-2c0e126b
    - Total Cloud Cover(TCC) => unit % => @p:nrel:r:20ed5e0a-acc8beff
    - Wind Speed at 19’ (5.7 meter) (WS) => unit mph => @p:nrel:r:20ed5df2-fd2eecc5
    

In [106]:
root_url = 'https://internal-apis.nrel.gov/intelligentcampus/hisRead?id='
reference_id = ['@p:nrel:r:225918db-bfbda16a','@p:nrel:r:20ed5e0a-275dbdc2','@p:nrel:r:20ed5e0a-53e174aa',
                '@p:nrel:r:20ed5e0a-fe755c80','@p:nrel:r:20ed5df2-2c0e126b','@p:nrel:r:20ed5e0a-acc8beff',
                '@p:nrel:r:20ed5df2-fd2eecc5']
date_range = '&range=\"2018-08-31%2c2018-09-06\"'
feature_name = ['EC','RH','BP','DBT','GHI','TCC','WS']

response_dict = {}
for i in range(len(reference_id)):
    response_dict['resp_'+feature_name[i]] = requests.get(root_url+reference_id[i]+date_range)
    if response_dict['resp_'+feature_name[i]].status_code == 200:
        pass
    else:
        print("response from {} is not getting fetched from API".format(feature_name[i]))

In [107]:
EC = response_dict['resp_EC'].content.decode('utf-8').split("\n")
EC = EC[2:]
EC = filter(None, EC)

In [108]:
def str_split(row):
    time_val = row.split(",")[0].strip(" Denver")
    energy_val = row.split(",")[1]
    energy_val = re.sub('[kwh%RHmbar°FW/m²_irrp]','', energy_val)
    return (time_val, float(energy_val))

EC = list(map(str_split, EC))
EC = list(zip(*EC))
EC_dt, EC_value = EC[0], EC[1]

In [109]:
def date_parser(row):
    parsed = parse(row)
    datetime_var = parsed.strftime(format='%m-%d-%y %H:%M:%S')
    date = parsed.date()
    time = parsed.time()
    return (datetime_var ,date, time)

EC_dt_parsed = list(map(date_parser, EC_dt))
EC_dt_parsed = list(zip(*EC_dt_parsed))
EC_datetime, EC_date, EC_time = EC_dt_parsed[0], EC_dt_parsed[1], EC_dt_parsed[2]

In [110]:
feature_name = ['EC','RH','BP','DBT','GHI','TCC','WS']
parsed_dict = {}
for i in range(len(feature_name)):
    parsed_dict[feature_name[i]] = response_dict['resp_'+feature_name[i]].content.decode('utf-8').split("\n")
    parsed_dict[feature_name[i]] = parsed_dict[feature_name[i]][2:]
    parsed_dict[feature_name[i]] = filter(None, parsed_dict[feature_name[i]])